In [43]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

def data_cleaning(ysg_book):
    book_info = ['도서명', '저자', 'ISBN', '권']

    # NaN인 열을 삭제한다
    ysg_book2 = ysg_book.dropna(axis=1, how='all')

    # '도서명', '저자', 'ISBN', '권', '대출건수'을 추출해서 count_df 데이터프레임을 만든다
    count_df = ysg_book2[['도서명', '저자', 'ISBN', '권', '대출건수']]

    # '도서명', '저자', 'ISBN', '권'을 기준으로 대출건수를 groupby한다
    loac_count = count_df.groupby(by=book_info, dropna=False).sum()

    # ysg_book2에서 중복된 행을 제외하고 고유한 행만 추출하여 복사한다
    dup_rows = ysg_book2.duplicated(subset=book_info)
    unique_rows = ~dup_rows
    ysg_book3 = ysg_book2[unique_rows].copy()

    # '도서명', '저자', 'ISBN', '권'을 index로 설정한다
    ysg_book3.set_index(book_info, inplace=True)
    
    # loac_count로 ysg_book3를 업데이트한다
    ysg_book3.update(loac_count)

    # 인덱스를 리셋한다
    ysg_book4 = ysg_book3.reset_index()

    # ysg_book4의 열을 ysg_book2 열로 설정한다
    ysg_book4 = ysg_book4[ysg_book2.columns]

    return ysg_book4

def get_book_info(row):
    title = row['도서명']
    author = row['저자']
    pub = row['출판사']
    year = row['발행년도']
    url = 'http://www.yes24.com/Product/Search?domain=ALL&query={}'
    r = requests.get(url.format(row['ISBN']))
    soup = BeautifulSoup(r.text, 'html.parser')
    try:
        # '도서명'이 NaN 이라면 yes24에서 도서명을 찾아서 넣는다
        if pd.isna(title):
            title = soup.find('a', attrs={'class':'gd_name'}).get_text()
    except AttributeError:
        pass

    try:
         # '저자'가 NaN 이라면 yes24에서 저자를 찾아서 넣는다
        if pd.isna(author):
            authors = soup.find('span', attrs={'class':'info_auth'}).find_all('a')
            author_list = [auth.get_text() for auth in authors]
            author = ','.join(author_list)
    except AttributeError:
        pass

    try:
         # '출판사'가 NaN 이라면 yes24에서 출판사를 찾아서 넣는다
        if pd.isna(pub):
            pub = soup.find('span', attrs={'class':'info_pub'}).find('a').get_text()
    except AttributeError:
        pass
    
    try:
         # '발행년도'가 -1 이라면 yes24에서 발행년도를 찾아서 넣는다
        if year == -1:
            year_str = soup.find('span', attrs={'class':'info_date'}).get_text()
            year = re.findall(r'\d{4}', year_str)[0]
    except AttributeError:
        pass

    return title, author, pub, year


def data_fixing(ysg_book1):
    # ysg_book1의 NaN인 열은 삭제하고, 필요한 열만 추출한다
    ysg_book1 = data_cleaning(ysg_book1)

    # ysg_book1의 '도서권수'와 '대출건수'를 int32로 변환
    ysg_book1 = ysg_book1.astype({'도서권수':'int32', '대출건수':'int32'})

    # ysg_book1의 '세트 ISBN'가 NaN 이라면 ''을 넣는다
    ysg_book1.loc[ysg_book1['세트 ISBN'].isna(), '세트 ISBN'] = ''

    # ysg_book1의 '발행년도'를 정규식으로 발행년도 4자리만 추출하고 다른 값이 있다면 -1을 넣는다
    ysg_book2 = ysg_book1.replace({'발행년도':'.*(\d{4}).*'}, r'\1', regex=True)
    ysg_book2.loc[ysg_book2['발행년도'].str.contains('\D', na=True), '발행년도'] = -1

    # ysg_book2의 '발행년도'를 int32로 변환
    ysg_book2 = ysg_book2.astype({'발행년도':'int32'})

    # ysg_book2의 '발행년도'가 4000년 이상인이라면 2333년 뺀다
    dangun_rows = ysg_book2['발행년도'].gt(4000)
    ysg_book2.loc[dangun_rows, '발행년도'] = ysg_book2.loc[dangun_rows, '발행년도'] - 2333

    # ysg_book2의 '발행년도'가 4000년 이상이라면 -1을 넣는다
    ysg_book2.loc[ysg_book2['발행년도'].gt(4000), '발행년도'] = -1

    # ysg_book2의 '발행년도'가 0~1900년 사이라면 -1을 넣는다
    ysg_book2.loc[ysg_book2['발행년도'].gt(0) & ysg_book2['발행년도'].lt(1900), '발행년도'] = -1

    # ysg_book2의 '도서명', '저자', '출판사'가 NaN 이거나 '발행년도'가 -1 이라면 
    # yes24에서 '도서명', '저자', '출판사', '발행년도'를 찾아와서 넣는다
    na_rows = ysg_book2['도서명'].isna() | ysg_book2['저자'].isna() | ysg_book2['출판사'].isna() | ysg_book2['발행년도'].eq(-1)
    updated_sample = ysg_book2[na_rows].apply(get_book_info, axis=1, result_type='expand')

    # updated_sample에 컬럼명을 '도서명', '저자', '출판사', '발행년도'으로 바꾼다
    updated_sample.columns = ['도서명', '저자', '출판사', '발행년도']

    # ysg_book2를 updated_sample로 업데이트한다
    ysg_book2.update(updated_sample)

    # ysg_book2의 '도서명', '저자', '출판사'가 NaN이고 '발행년도'가 -1인 행을 지운다 
    ysg_book6 = ysg_book2.dropna(subset=['도서명','저자','출판사'])
    ysg_book6 = ysg_book6[ysg_book6['발행년도'] != -1]
    
    return ysg_book6 

# 도서정보를 데이터프레임으로 불러오기
ysg_book1 = pd.read_csv('ysg_book.csv', encoding='cp949')

ysg_book6 = data_fixing(ysg_book1)
ysg_book6

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,헤어질 결심 각본,"정서경,박찬욱 지음",을유문화사,2022,9788932474755,,0,NaN,812.66,1,0,2023-03-10
1,2,화이트 러시,히가시노 게이고 지음;민경욱 옮김,㈜소미미디어,2023,9791138415477,,0,NaN,833.60,1,1,2023-03-10
2,3,크리스마스 타일,김금희 지음,창비,2022,9788936438890,,0,NaN,813.60,1,1,2023-03-10
3,4,체셔 크로싱,앤디 위어 글;사라 앤더슨 그림;황석희 옮김,알에이치코리아(RHK),2022,9788925577319,,0,NaN,843.60,1,0,2023-03-10
4,5,자발적 방관육아,최은아 지음,쌤앤파커스,2023,9791165346805,,0,NaN,598.10,1,1,2023-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14647,14776,(고정관념을 날려버리는) 5분 철학 오프너,줄리아 드 퓌네스 지음;이나무 옮김,이숲,2012,9788994228150,,0,NaN,104.00,1,1,2014-02-11
14648,14777,(겸손의 미덕으로 미래를 바꾼) 후진타오 이야기,박근형 지음,명진출판,2012,9788976776235,,0,8,990.80,1,0,2014-02-11
14649,14778,앤디 워홀 이야기,아서 단토 지음;박선령 옮김;이혜경 엮음,명진출판,2012,9788976776280,,0,10,990.80,1,0,2014-02-11
14650,14779,4001,신정아 지음,사월의책,2011,9788996461067,,0,NaN,818.00,1,2,2014-02-11
